# Deteccao de Melanoma por Machine Learning

Montando driver

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Imports

In [ ]:
import os
import cv2
import numpy as np
import random as rn
import pandas as pd
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

Preparacao do dataset
(Codigo executado antes da importacao do dataset ao colab)

In [ ]:
import tensorflow as tf
import keras
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from tensorflow.keras.activations import elu
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputSpec, Layer
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout,MaxPooling2D,BatchNormalization,GlobalMaxPooling2D

# set seed for reproducability
seed=1234
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ["PYTHONHASHSEED"]=str(seed)

Especificando o Modelo

In [ ]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(256,256,3),include_top=False,weights="imagenet")

for layer in base_model.layers:
    layer.trainable=True


# Build Model according to your output
def build_model():
    model=Sequential()
    model.add(base_model) 
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.3))  # change dropout get better result
    model.add(Dense(1,activation="sigmoid")) # output is 0 or 1 binary 
    
    # now compile model
    # we are using adam you can use other to optimize better
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005,beta_1=0.9,beta_2=0.999,amsgrad=False)
    # This is important part of model 
    # Number of positive image is less than number of negative image
    # we use AUC metrics
    metrics=tf.keras.metrics.AUC(name="auc")
    model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=metrics)
    print(model.summary())
    return model

model=build_model()

Read CSV

In [ ]:
df=pd.read_csv("/content/gdrive/My Drive/MelanomaData/train.csv")
df

In [ ]:
a,b=np.unique(df["target"],return_counts=True)
a
b

Separar dataset em treino e validacao

In [ ]:
from sklearn.model_selection import train_test_split


train,valid=train_test_split(df,test_size=0.2)
train,test=train_test_split(train,test_size=0.01)

train["image_name_jpg"]=train["image_name"]+".jpg"
test["image_name_jpg"]=test["image_name"]+".jpg"
valid["image_name_jpg"]=valid["image_name"]+".jpg"

train["target"]=train["target"].astype(str)
test["target"]=test["target"].astype(str)
valid["target"]=valid["target"].astype(str)

Processando data de treinamento:


In [ ]:
# now create train_generator and validation_generator
# you can add other augmentation example vertical_flip, random cropping ,etc to get bettter accuracy
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    horizontal_flip=True
    )
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255) # we have to divide by 255 for testing in android app


train_generator =train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="/content/gdrive/My Drive/MelanomaData/train/",
    x_col="image_name_jpg", # name +".jpg"
    y_col="target",
    target_size=(256,256),
    batch_size=32,
    class_mode="binary"
)

Processando data de Validacao

In [ ]:
validation_generator =test_datagen.flow_from_dataframe(
    dataframe=valid,
    directory="/content/gdrive/My Drive/MelanomaData/train/",
    x_col="image_name_jpg",
    y_col="target",
    target_size=(256,256),
    batch_size=16,
    class_mode="binary"
)

Definindo Modelo de Treinamento

In [ ]:
model.fit_generator(
    train_generator,
    epochs=3, #you have set or use early stopping to stop before overfiting
    shuffle=True,
    validation_data=validation_generator
    # can also define callback to save model when gets best accuracy
)

In [ ]:
#test saving

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)